In [1]:
import sys
import random
import PySide6
from PySide6 import QtCore, QtWidgets, QtGui
from PySide6.QtWidgets import *
from PySide6.QtCore import Qt
from PySide6.QtGui import QPalette, QColor
import numpy as np
import pandas as pd


ModuleNotFoundError: No module named 'PySide6'

In [20]:
data=None

try:
    data=pd.read_feather("data/stage3_partial.feather")
except:
    try:
        print("Loading backup, main data corrupted.")
        data=pd.read_feather("data/stage3_partial.backup.feather")
    except:
        print("Loading stage 1, main data & backup corrupted.")
        data=pd.read_feather("data/stage2.feather")


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18901 entries, 0 to 19287
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   path       18901 non-null  object 
 1   class      18901 non-null  object 
 2   predicted  18901 non-null  object 
 3   certainty  18901 non-null  float32
dtypes: float32(1), object(3)
memory usage: 664.5+ KB


In [22]:
def save(is_backup=False):
    data.to_feather(f"data/stage3_partial{'.backup' if is_backup else ''}.feather")
save(True)

In [23]:
data[data["class"]==""].index

Index([  395,   397,   399,   400,   402,   403,   405,   406,   407,   408,
       ...
       19278, 19279, 19280, 19281, 19282, 19283, 19284, 19285, 19286, 19287],
      dtype='int64', length=1420)

In [63]:
def get_next():
    idx=data[data["class"]==""].index
    if(idx.shape[0]==0):
        return None, 0, None
    return idx[0],idx.shape[0], data.loc[idx[0]]

In [64]:
next_idx,remainder,row=get_next()

In [65]:
def safe_update(idx, value):
    save(True)
    data.loc[idx,"class"]=value
    save(False) 

In [66]:
keys = ["UNKNOWN", "INACTIVE", "WITH_FUME", "WITH_EXPLOSION", "WITH_FUME_AND_EXPLOSION"]
labels = {
    keys[0]:"No es el volcán",
    keys[1]:"El volcán está inactivo",
    keys[2]:"El volcán tiene una fumarola",
    keys[3]:"El volcán hace explosión",
    keys[4]:"El volcán tiene una fumarola y hace explosión",
}

In [67]:
window=None

In [68]:
class ImageDisplay(QWidget):

    def loadpixmap(self, path):
        self.pixmap=QtGui.QPixmap(path)#.scaledToHeight(self.geometry().width())
        self.label.setPixmap(self.pixmap)
        
    def __init__(self, path = ""):
        super().__init__()
        label=QLabel()
        label.setAlignment(Qt.Alignment.AlignCenter)
        label.setScaledContents(True)
        self.label=label
        self.loadpixmap(path)
        
        self.layout=QGridLayout()
        self.layout.addWidget(label, 0,0)
        self.setLayout(self.layout)

In [69]:
class DataLayout(QWidget):

    def button_pressed(self, key):
        global next_idx,remainder,row
        """
        safe_update(next_idx, self.values)
        next_idx,remainder,row=get_next()
        window.image.loadpixmap(row["path"])
        #for slider in self.sliders:
        #    slider.setValue(0)
        self.values=[]
        for slider in self.sliders:
            self.values.append(slider.value())
        self.progress.setText(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%) Suggested: {row['predicted']} {row['certainty']*100:.02F}%")
        #window.close()
        """
        safe_update(next_idx,key)
        next_idx,remainder,row=get_next()
        window.image.loadpixmap(row["path"])
        self.progress.setText(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%) Suggested: {row['predicted']} {row['certainty']*100:.02F}%")


    def get_callback(self,key):
        return lambda: self.button_pressed(key)
    
    def __init__(self):
        super().__init__()
        self.setMinimumWidth(600)
        self.setMaximumWidth(600)
        layout=QVBoxLayout()

        self.progress = QLabel(f"Remaining: {remainder} ({100*(remainder/data.shape[0]):.02F}%) Suggested: {row['predicted']} {row['certainty']*100:.02F}%")
        self.progress.setMaximumHeight(50)
        self.progress.setAlignment(Qt.Alignment.AlignCenter)
        layout.addWidget(self.progress)
        
        for i,key in enumerate(keys):
            button=QPushButton(labels[key])
            button.setMinimumHeight(50)
            # set specific key, force it to stay fixed each loop
            button.clicked.connect(self.get_callback(key))
            layout.addWidget(button)
            
        self.setLayout(layout)

In [70]:
class MainWindow(QMainWindow):

    def __init__(self):
        super(MainWindow, self).__init__()

        self.setWindowTitle("Classifier")

        layout = QHBoxLayout()
        left = ImageDisplay(row["path"])
        right= DataLayout()

        self.data=right
        self.image=left

        layout.addWidget(left)
        layout.addWidget(right)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)

In [71]:
if __name__ == "__main__" and remainder>0:
    if not QtWidgets.QApplication.instance():
        app = QtWidgets.QApplication([])
    else:
        app = QtWidgets.QApplication.instance()
    font=QtGui.QFont("Segoe UI", 14)
    app.setFont(font)
    app.setStyle(QStyleFactory.create("windows"))
    window = MainWindow()
    window.showMaximized()

    app.exec()

TypeError: 'NoneType' object is not subscriptable

KeyError: None

In [60]:
data.groupby(by="predicted")["certainty"].describe()

,count,mean,std,min,25%,50%,75%,max
predicted,,,,,,,,
EXP,241.0,0.902565,0.151383,0.413489,0.865470,0.990975,0.999591,0.999999
EXP+FUM,7100.0,0.985119,0.062001,0.364781,0.999202,0.999958,0.999996,1.000000
FUM,8325.0,0.984883,0.061563,0.374945,0.999083,0.999955,0.999997,1.000000
INA,2734.0,0.981509,0.065010,0.444901,0.998364,0.999956,0.999998,1.000000
UNK,501.0,0.929717,0.133045,0.354192,0.955269,0.991641,0.997389,0.998998


In [78]:
data[data["class"]==""]

,path,class,predicted,certainty


In [79]:
data

,path,class,predicted,certainty
0.0,images/2006/jun/p0622063.jpg,UNKNOWN,UNK,0.998998
2.0,images/2013/sep/p0926132.jpg,UNKNOWN,UNK,0.998995
3.0,images/2019/ene/p0120193.jpg,UNKNOWN,UNK,0.998994
4.0,images/2018/jul/p0723181.jpg,UNKNOWN,UNK,0.998969
5.0,images/2015/mar/p0329153.jpg,UNKNOWN,UNK,0.998967
...,...,...,...,...
19283.0,images/2015/may/p0530155.jpg,WITH_EXPLOSION,EXP,0.489222
19284.0,images/2016/oct/p1028164.jpg,WITH_EXPLOSION,EXP,0.461512
19285.0,images/2018/dic/p1231184.jpg,WITH_EXPLOSION,EXP,0.459877
19286.0,images/2018/jun/p0603184.jpg,WITH_EXPLOSION,EXP,0.421862


In [80]:
save(True)

In [77]:
save(False)